# 8. Clustering

Dans ce notebook nous appliquons des algorithmes de clustering sur des données synthétiques.

In [ ]:
import matplotlib.pyplot as plt

from sklearn.datasets import make_blobs, make_circles, make_moons # for generating experimental data
from sklearn.cluster import KMeans 
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

%matplotlib inline

## 8.1 Création des datasets synthétiques

Les fonctions `make_blobs` et apparentées permettent de créer des jeux de données synthétiques, dans notre cas des points dans un espace à deux dimensions qui respectent un ensemble de contraintes (des nuages de points, des cercles de points ...) :

In [ ]:
X_blobs, y_blobs = make_blobs(n_samples=5000,
                              n_features=2,
                              centers=3,
                              random_state=170)
X_circles, y_circles = make_circles(n_samples=1000,
                                    factor=0.3,
                                    noise=0.1)
X_moons, y_moons = make_moons(n_samples=1000,
                              noise=.05)
X_varied, y_varied = make_blobs(n_samples=1000,
                                random_state=8,
                                cluster_std=[1.0, 2.5, 0.5])

La fonction `plot_clusters` permet d'afficher ces datasets (et les clusters, comme nous le verrons plus loin). Utilisez cette fonction pour afficher nos différents datasets :

In [ ]:
def plot_clusters(X, labels=None, centers=None):
    plt.figure(figsize=(15, 7))
    plt.scatter(X[:,0], X[:,1], c=labels, edgecolors='k')
    if centers is not None:
        plt.scatter(
            centers[:,0], 
            centers[:,1], 
            c='w', marker='x', s=5, linewidth=30
        )

In [ ]:
plot_clusters(X_blobs)

In [ ]:
plot_clusters(X_circles)

In [ ]:
plot_clusters(X_moons)

In [ ]:
plot_clusters(X_varied)

## 8.2 K-means

Nous allons appliquer l'algorithme K-means sur notre dataset `X_blobs` en spécifiant que nous souhaitons obtenir 3 clusters.

Créez une instance (que vous nommerez `kmeans`) de la classe [`sklearn.cluster.KMeans`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) en spécifiant les paramètres :
* `n_clusters = 3`
* `n_init = 3`
* `init = 'random'`
* `tol = 1e-4`
* `random_state = 170`
* `verbose = True`

En vous aidant de la documentation de cette classe sur le site de scikit-learn, décrivez l'effet de chacun des paramètres spécifiés ci-dessus.

In [ ]:
kmeans = KMeans(n_clusters=3, n_init=3, init='random', tol=1e-4, random_state=170, verbose=True)

Appelez la méthode `fit_predict` en lui fournissant comme seul paramètre notre dataset `X_blobs` et stockez le résultat dans une variable `y_pred` :

In [ ]:
y_pred = kmeans.fit_predict(X_blobs)

Affichez le contenu des attributs `labels_` et `cluster_centers_` de `kmeans`. A quoi correspondent ces attributs ?

In [ ]:
print(kmeans.labels_)
print(kmeans.cluster_centers_)
print(kmeans.inertia_)

Ce sont la liste des clusters de chaque point de notre dataset ainsi que les coordonnées des centroids.

Utilisez la fonction `plot_clusters` pour afficher le résultat de ce clustering sur le dataset `X_blobs` (vous devez spécifier tous les paramètres de cette fonction) :

In [ ]:
plot_clusters(X_blobs, kmeans.labels_, kmeans.cluster_centers_)

## 8.3 Comportement de K-means

En contraignant incrémentalement le nombre d'itération de K-means (et en fixant un `random_state` identique), nous pouvons visualiser les déplacements des centroids.

Affichez le résultat d'un clustering K-means en variant le nombre d'itérations de 1 à 11 et commentez les résultats obtenus :

In [ ]:
for i in range(1, 11):
    kmeans = KMeans(n_clusters=3, n_init=1, init='random', random_state=7, max_iter=i).fit(X_blobs)
    plot_clusters(X_blobs, kmeans.labels_, kmeans.cluster_centers_)

Nous observons que les centroids se déplacent beaucoup entre les premières itérations. Et qu'il n'y a probablement pas de variation de la position des centroids au dela de la 6ème itération. Pour s'en assurer, nous pourrions afficher les coordonnées des centroids à chaque itération. Nous pouvons aussi faire varier le `random_state` pour observer le comportement de K-means avec d'autres choix de centroids initiaux.

Nous avons fixé le paramètre `init` à `random` pour obtenir un choix aléatoire des centroids initiaux.
En utilisant `k-means++` (valeur par défaut pour le paramètre `init`) comme méthode d'initialisation des centroids nous pourrions converger plus rapidement vers une solution stable. Sur notre dataset synthétique, nous aurions les centroids idéalement positionnés dès la première itération.

## 8.4 Recherche du nombre de clusters optimal

Notre jeu de données synthétique `X_blobs` contient clairement 3 clusters, mais que se passe-t'il si nous précisons un autre nombre de clusters à K-means ?

Modifiez le nombre de clusters voulus et observez le résultat obtenu :

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42).fit(X_blobs)
plot_clusters(X_blobs, kmeans.labels_, kmeans.cluster_centers_)

Une méthode usuelles pour déterminer le nombre de clusters optimum obtenus avec K-means consiste à déterminer à partir de combien de clusters l'inertie de notre clustering cesse de varier significativement.

Constuisez une liste contenant l'inertie pour des clusterings de tailles croissantes de 1 à 11 (vous pouvez utiliser la fonction [`range`](https://docs.python.org/fr/3/tutorial/controlflow.html#the-range-function) de Python pour créer une boucle de tous les entiers entre 1 et 11) :

In [ ]:
inertia = []
clusters_sizes = range(1, 11)
for n in clusters_sizes:
    kmeans = KMeans(n_clusters=n, random_state=42).fit(X_blobs)
    inertia.append(kmeans.inertia_)

La fonction suivante affiche l'intertie en fonction d'un nombre de clusters. Utilisez cette fonction pour afficher l'inertie de 1 à 11 clusters :

In [ ]:
def plot_elbow(clusters_sizes, inertia):
    plt.figure(1, figsize=(15, 6))
    plt.plot(clusters_sizes, inertia)
    plt.title('The Elbow Method')
    plt.xlabel('Number of clusters')
    plt.ylabel('inertia')
    plt.show()

In [ ]:
plot_elbow(clusters_sizes, inertia)

## 8.5 Limites de K-means

K-means fonctionne correctement sur des jeux de données possédant des clusters *sphériques*, si nous appliquons cet algorithme sur nos autres jeux de données synthétiques, les résultats ne seront pas aussi convaincants.

Appliquez un clustering K-means sur le dataset `X_circles` avec 2 clusters, puis affichez l'_elbow curve_ pour 1 à 21 clusters :

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42).fit(X_circles)
plot_clusters(X_circles, kmeans.labels_, kmeans.cluster_centers_)

In [ ]:
inertia = []
clusters_sizes = range(1, 21)
for n in clusters_sizes:
    kmeans = KMeans(n_clusters=n, random_state=42).fit(X_circles)
    inertia.append(kmeans.inertia_)

In [ ]:
plot_elbow(clusters_sizes, inertia)

Une solution pour contourner cette limitation de K-means consiste à augmenter le nombre de clusters. Appliquez K-means avec 6 clusters sur `X_circles` :

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=42).fit(X_circles)
plot_clusters(X_circles, kmeans.labels_, kmeans.cluster_centers_)

## 8.6 DBScan

Nous utilisons maintenant l'algorithme basé sur la densité DBScan sur ce même dataset. Les paramètres les plus importants pour cet algorithme sont :
- epsilon (`eps`) qui définit la distance maximale pour considérer que deux points sont voisins
- `min_samples` qui définit le nombre de voisins nécéssaires pour qu'un point soit considérer comme un *core point*

Attention : il faut normaliser les données en entrée de DBScan avec [`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

Normalisez le dataset `X_circle`, créez une instance de [`sklearn.cluster.DBSCAN`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) avec les paramètres `eps=0.2` et `min_samples=30`, appelez la méthode `fit_predict` de cette instance sur `X_circles` normalisé, puis affichez le résultat de ce clustering.

In [ ]:
X_scaled = StandardScaler().fit_transform(X_circles)
y_pred = DBSCAN(eps=0.2, min_samples=10).fit_predict(X_scaled)
plot_clusters(X_circles,y_pred)

In [ ]:
y_pred

Appliquez DBSCAN (avec `eps=0.5` et `min_samples=30`) et K-means (avec `nclusters=2`) sur le dataset `X_moons` (normalisez ce dataset pour DBSCAN). Quel est la méthode qui fournit le meilleur résultat ? Pourquoi la moins bonne n'arrive pas à regrouper nos deux arcs de cercles ?

In [ ]:
X_scaled = StandardScaler().fit_transform(X_moons)
y_pred = DBSCAN(eps=0.5, min_samples=30).fit_predict(X_scaled)
plot_clusters(X_moons,y_pred)

In [ ]:
kmeans = KMeans(n_clusters=50, random_state=42).fit(X_moons)
plot_clusters(X_moons, kmeans.labels_, kmeans.cluster_centers_)

K-means se basant sur la distance entre les points de données, cet algorithme ne peut pas séparer correctement les individus de  `X_moons`.

## 8.7 Limitation de DBScan

Comparez le clustering obtenu avec DBSCAN et K-means sur le dataset `X_varied`. Calculez le _silhouette_score_ avec la fonction [`sklearn.metrics.silhouette_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html) pour les deux clusterings. Quelle est la meilleure méthode ? Pourquoi la moins bonne méthode n'arrive pas à faire ressortir nos 3 clusters ?

In [ ]:
X_scaled = StandardScaler().fit_transform(X_varied)
y_pred = DBSCAN(eps=0.5, min_samples=30).fit_predict(X_scaled)
plot_clusters(X_varied,y_pred)
silhouette_score(X_varied, y_pred)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42).fit(X_varied)
plot_clusters(X_varied, kmeans.labels_, kmeans.cluster_centers_)
silhouette_score(X_varied, kmeans.fit_predict(X_varied))

In [ ]:
inertia = []
clusters_sizes = range(1, 11)
for n in clusters_sizes:
    kmeans = KMeans(n_clusters=n, random_state=42).fit(X_varied)
    inertia.append(kmeans.inertia_)

In [ ]:
plot_elbow(clusters_sizes, inertia)

DBScan est moins efficace si nous avons des clusters de densités différentes. Le dataset `X_varied` présente la particularité de contenir trois clusters de densités différentes. DBScan appliqué sur ce dataset ne peut pas détecter les trois clusters, en revanche K-means arrive bien à les séparer et l'*elbow method* met en évidence qu'il y a trois clusters dans ce dataset.